<a href="https://colab.research.google.com/github/mszzukowski/dw_matrix_rad_sign/blob/master/day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import os
import datetime

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

In [2]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_rad_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_rad_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [6]:
model = Sequential([
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=input_shape),

                    Flatten(),

                    Dense(num_classes, activation='softmax'),
])

# model.summary()

model.compile(loss='categorical_crossentropy', optimizer='Adam' ,metrics=['accuracy'])
model.fit(X_train, y_train)

1088/1088 [==============================] - 4s 3ms/step - loss: 9.3992 - accuracy: 0.7725


In [0]:
def get_cnn_c1(input_shape, num_classes):
  return Sequential([
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=input_shape),

                    Flatten(),

                    Dense(num_classes, activation='softmax'),
  ])

def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam' ,metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
    )

  
  return model

In [8]:
model = get_cnn_c1(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

Epoch 1/5
272/272 [==============================] - 3s 12ms/step - loss: 37.8692 - accuracy: 0.7625 - val_loss: 0.2037 - val_accuracy: 0.9554
Epoch 2/5
272/272 [==============================] - 3s 11ms/step - loss: 0.1812 - accuracy: 0.9581 - val_loss: 0.1184 - val_accuracy: 0.9731
Epoch 3/5
272/272 [==============================] - 3s 11ms/step - loss: 0.0996 - accuracy: 0.9770 - val_loss: 0.0895 - val_accuracy: 0.9791
Epoch 4/5
272/272 [==============================] - 3s 11ms/step - loss: 0.0874 - accuracy: 0.9790 - val_loss: 0.0768 - val_accuracy: 0.9827
Epoch 5/5
272/272 [==============================] - 3s 11ms/step - loss: 0.1718 - accuracy: 0.9634 - val_loss: 0.0673 - val_accuracy: 0.9857


In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [10]:
predict(model_trained, X_test, y_test)

0.8369614512471655

In [0]:
df = pd.read_csv('data/signnames.csv')
labels_dict = df.to_dict()['b']

Archicture engenering

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)

  return predict(model_trained, X_test, y_test)

In [13]:
def get_cnn_v2(input_shape, num_classes):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),

                    Flatten(),
                    
                    Dense(1024, activation='relu'),
                    Dropout(0.5),

                    Dense(num_classes, activation='softmax'),
  ])


train_and_predict(get_cnn_v2(input_shape, num_classes))


Epoch 1/5
272/272 [==============================] - 3s 11ms/step - loss: 6.5025 - accuracy: 0.1876 - val_loss: 2.3333 - val_accuracy: 0.3945
Epoch 2/5
272/272 [==============================] - 3s 10ms/step - loss: 2.0820 - accuracy: 0.4157 - val_loss: 1.2135 - val_accuracy: 0.6796
Epoch 3/5
272/272 [==============================] - 3s 11ms/step - loss: 1.2375 - accuracy: 0.6274 - val_loss: 0.5164 - val_accuracy: 0.8796
Epoch 4/5
272/272 [==============================] - 3s 11ms/step - loss: 0.7860 - accuracy: 0.7653 - val_loss: 0.2878 - val_accuracy: 0.9359
Epoch 5/5
272/272 [==============================] - 3s 11ms/step - loss: 0.5598 - accuracy: 0.8326 - val_loss: 0.1518 - val_accuracy: 0.9730


0.8884353741496599

In [14]:
def get_cnn_v3(input_shape, num_classes):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),

                    Flatten(),
                    
                    Dense(1024, activation='relu'),
                    Dropout(0.5),

                    Dense(num_classes, activation='softmax'),
  ])


train_and_predict(get_cnn_v3(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 4s 16ms/step - loss: 2.7207 - accuracy: 0.4060 - val_loss: 0.5357 - val_accuracy: 0.8790
Epoch 2/5
272/272 [==============================] - 4s 15ms/step - loss: 0.6514 - accuracy: 0.8091 - val_loss: 0.1973 - val_accuracy: 0.9597
Epoch 3/5
272/272 [==============================] - 4s 15ms/step - loss: 0.3344 - accuracy: 0.9042 - val_loss: 0.0736 - val_accuracy: 0.9860
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2099 - accuracy: 0.9395 - val_loss: 0.0345 - val_accuracy: 0.9916
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.1587 - accuracy: 0.9533 - val_loss: 0.0249 - val_accuracy: 0.9930


0.9675736961451247

In [15]:
def get_cnn_v4(input_shape, num_classes):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),

                    Flatten(),
                    
                    Dense(1024, activation='relu'),
                    Dropout(0.5),

                    Dense(num_classes, activation='softmax'),
  ])


train_and_predict(get_cnn_v4(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 5s 18ms/step - loss: 2.9843 - accuracy: 0.2079 - val_loss: 1.5616 - val_accuracy: 0.4747
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 1.3508 - accuracy: 0.5653 - val_loss: 0.3858 - val_accuracy: 0.8883
Epoch 3/5
272/272 [==============================] - 5s 17ms/step - loss: 0.5631 - accuracy: 0.8232 - val_loss: 0.1169 - val_accuracy: 0.9684
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.3203 - accuracy: 0.9006 - val_loss: 0.0622 - val_accuracy: 0.9810
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2275 - accuracy: 0.9314 - val_loss: 0.0391 - val_accuracy: 0.9902


0.971655328798186

In [17]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),

                    Flatten(),
                    
                    Dense(1024, activation='relu'),
                    Dropout(0.3),

                    Dense(1024, activation='relu'),
                    Dropout(0.3),

                    Dense(num_classes, activation='softmax'),
  ])


train_and_predict(get_cnn_v5(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 5s 18ms/step - loss: 2.8608 - accuracy: 0.2389 - val_loss: 1.1017 - val_accuracy: 0.6594
Epoch 2/5
272/272 [==============================] - 5s 18ms/step - loss: 0.9525 - accuracy: 0.7024 - val_loss: 0.2406 - val_accuracy: 0.9280
Epoch 3/5
272/272 [==============================] - 5s 18ms/step - loss: 0.4225 - accuracy: 0.8697 - val_loss: 0.1043 - val_accuracy: 0.9718
Epoch 4/5
272/272 [==============================] - 5s 18ms/step - loss: 0.2816 - accuracy: 0.9165 - val_loss: 0.0480 - val_accuracy: 0.9866
Epoch 5/5
272/272 [==============================] - 5s 18ms/step - loss: 0.1939 - accuracy: 0.9442 - val_loss: 0.0459 - val_accuracy: 0.9854


0.9530612244897959

In [0]:
X_train_gray = color.rgb2gray( X_train).reshape(-1, 32, 32, 1)
X_test_gray = color.rgb2gray( X_test).reshape(-1, 32, 32, 1)

In [27]:
model = get_cnn_v5((32,32,1), num_classes)
params_fit = {
    'epochs': 25
}
model_trained = train_model(model, X_train_gray, y_train, params_fit)
predict(model_trained, X_test_gray, y_test)

ERROR! Session/line number was not unique in database. History logging moved to new session 59
Epoch 1/25
272/272 [==============================] - 5s 19ms/step - loss: 2.7747 - accuracy: 0.2273 - val_loss: 0.7612 - val_accuracy: 0.7461
Epoch 2/25
272/272 [==============================] - 5s 18ms/step - loss: 0.5650 - accuracy: 0.8131 - val_loss: 0.1240 - val_accuracy: 0.9666
Epoch 3/25
272/272 [==============================] - 5s 18ms/step - loss: 0.2366 - accuracy: 0.9243 - val_loss: 0.0575 - val_accuracy: 0.9824
Epoch 4/25
272/272 [==============================] - 5s 18ms/step - loss: 0.1503 - accuracy: 0.9534 - val_loss: 0.0298 - val_accuracy: 0.9922
Epoch 5/25
272/272 [==============================] - 5s 18ms/step - loss: 0.1161 - accuracy: 0.9634 - val_loss: 0.0240 - val_accuracy: 0.9936
Epoch 6/25
272/272 [==============================] - 5s 18ms/step - loss: 0.0947 - accuracy: 0.9706 - val_loss: 0.0194 - val_accuracy: 0.9943
Epoch 7/25
272/272 [===========================

0.9836734693877551

In [0]:
# %tensorboard --logdir logs

In [11]:
model = get_cnn_c2(input_shape, num_classes)
params_fit = {
    'epochs': 25
}
logdir = train_model(model, X_train, y_train, params_fit)

Epoch 1/25
272/272 [==============================] - 3s 12ms/step - loss: 6.9235 - accuracy: 0.1881 - val_loss: 1.9751 - val_accuracy: 0.4685
Epoch 2/25
272/272 [==============================] - 3s 11ms/step - loss: 1.5860 - accuracy: 0.5381 - val_loss: 0.7257 - val_accuracy: 0.8076
Epoch 3/25
272/272 [==============================] - 3s 11ms/step - loss: 0.9363 - accuracy: 0.7175 - val_loss: 0.3460 - val_accuracy: 0.9129
Epoch 4/25
272/272 [==============================] - 3s 11ms/step - loss: 0.6591 - accuracy: 0.7964 - val_loss: 0.2092 - val_accuracy: 0.9502
Epoch 5/25
272/272 [==============================] - 3s 11ms/step - loss: 0.5192 - accuracy: 0.8400 - val_loss: 0.1699 - val_accuracy: 0.9576
Epoch 6/25
272/272 [==============================] - 3s 11ms/step - loss: 0.4311 - accuracy: 0.8649 - val_loss: 0.1133 - val_accuracy: 0.9710
Epoch 7/25
272/272 [==============================] - 3s 11ms/step - loss: 0.3826 - accuracy: 0.8805 - val_loss: 0.0910 - val_accuracy: 0.9787